In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
import pickle
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D

Using TensorFlow backend.


In [8]:
emails=pd.read_csv("endava_email_tickets_ML.csv")

In [9]:
regexList = []
regexList += ['From:(.*)\r\n']  # from line
regexList += ['^[_a-z0-9-]+(\.[_a-z0-9-]+)*@[a-z0-9-]+(\.[a-z0-9-]+)*(\.[a-z]{2,4})$']
regexList += ['[\w\d\-\_\.]+ @ [\w\d\-\_\.]+']
regexList += ['Subject:']
regexList += ['[^a-zA-Z]']
emails['body'] = emails['body'].replace(regexList, ' ')

In [10]:
replace_list = {r"i'm": 'i am',
                r"'re": ' are',
                r"let’s": 'let us',
                r"'s":  ' is',
                r"'ve": ' have',
                r"can't": 'can not',
                r"cannot": 'can not',
                r"shan’t": 'shall not',
                r"n't": ' not',
                r"'d": ' would',
                r"'ll": ' will',
                r"'scuse": 'excuse',
                ',': ' ,',
                '.': ' .',
                '!': ' !',
                '?': ' ?',
                '\s+': ' '}

In [11]:
def clean_text(text):
    text = text.lower()    
    for s in replace_list:
        text = text.replace(s, replace_list[s])
    text = ' '.join(text.split())
    return text
X_body = emails['body'].apply(lambda p: clean_text(p))

In [12]:
Y_ticket_type = emails['ticket_type']

In [13]:
phrase_len = X_body.apply(lambda p: len(p.split(' ')))
max_phrase_len = phrase_len.max()

In [14]:
max_words = 5000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_body)
X = tokenizer.texts_to_sequences(X_body)
X = pad_sequences(X, maxlen = max_phrase_len)
Y = to_categorical(Y_ticket_type)

In [15]:
max_phrase_len

980

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)

In [32]:
#y_train[0]

In [17]:
model_lstm_ttype = Sequential()
model_lstm_ttype.add(Embedding(10000, 128, input_length = max_phrase_len))
model_lstm_ttype.add(SpatialDropout1D(0.3))
model_lstm_ttype.add(LSTM(128, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm_ttype.add(Dense(2, activation = 'softmax'))
model_lstm_ttype.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

W1211 20:28:35.278507  7872 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1211 20:28:35.729145  7872 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1211 20:28:35.734623  7872 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1211 20:28:35.824780  7872 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1211 20:28:35.852732  7872 deprecation.py:506

In [18]:
history = model_lstm_ttype.fit(
    X_train,
    y_train,
    validation_split = 0.3,
    epochs = 1,
    batch_size = 128)

W1211 20:29:00.916250  7872 deprecation.py:323] From C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 23788 samples, validate on 10196 samples
Epoch 1/1
23788/23788 [==============================] - 3905s 164ms/step - loss: 0.1244 - acc: 0.9470 - val_loss: 0.0174 - val_acc: 0.9948


In [19]:
accr_ttype = model_lstm_ttype.evaluate(X_test,y_test)

14565/14565 [==============================] - 991s 68ms/step


In [24]:
accr_ttype

[0.0171144532336215, 0.9939581187778922]

In [21]:
pickle.dump(model_lstm_ttype, open('model_ttype.pkl','wb'))

In [22]:
test=['icon dear please setup icon per icon engineers please let other details needed thanks lead']
test_tokens = tokenizer.texts_to_sequences(test)
test_sequence = pad_sequences(test_tokens, maxlen = max_phrase_len)

In [23]:
model = pickle.load(open('model_ttype.pkl','rb'))
print(np.argmax(model.predict(test_sequence)))

1
